In [1]:
import re
import os
from pathlib import Path
import warnings
from glob import glob
import math
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams, cycler
import matplotlib.lines as lines
from collections import OrderedDict
from matplotlib.legend import Legend
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import seaborn as sns
import scipy.integrate as integrate
#import scipy.special as special

#from sympy import elliptic_pi

import MDAnalysis as mda 
from MDAnalysis.coordinates.LAMMPS import DumpReader
from MDAnalysis import transformations as mdatransform
from MDAnalysis.analysis import diffusionmap, align, rms

import polyphys

/Users/amirhsi_mini/opt/anaconda3/envs/polyPhys/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")


In [3]:
class AmirDumpReader(DumpReader):
    
    format = 'AMIRLAMMPSDUMP'
    
    def __init__(self, filename, fractional=False, **kwargs):
        self.fractional = fractional
        super().__init__(filename, **kwargs)
        
    def _read_next_timestep(self):
        f = self._file
        ts = self.ts
        ts.frame += 1
        if ts.frame >= len(self):
            raise EOFError

        f.readline() # ITEM TIMESTEP
        step_num = int(f.readline())
        ts.data['step'] = step_num

        f.readline() # ITEM NUMBER OF ATOMS
        n_atoms = int(f.readline())
        if n_atoms != self.n_atoms:
            raise ValueError("Number of atoms in trajectory changed "
                             "this is not suported in MDAnalysis")

        triclinic = len(f.readline().split()) == 9  # ITEM BOX BOUNDS
        if triclinic:
            xlo, xhi, xy = map(float, f.readline().split())
            ylo, yhi, xz = map(float, f.readline().split())
            zlo, zhi, yz = map(float, f.readline().split())

            box = np.zeros((3, 3), dtype=np.float64)
            box[0] = xhi - xlo, 0.0, 0.0
            box[1] = xy, yhi - ylo, 0.0
            box[2] = xz, yz, zhi - zlo

            xlen, ylen, zlen, alpha, beta, gamma = mdamath.triclinic_box(*box)
        else:
            xlo, xhi = map(float, f.readline().split())
            ylo, yhi = map(float, f.readline().split())
            zlo, zhi = map(float, f.readline().split())
            xlen = xhi - xlo
            ylen = yhi - ylo
            zlen = zhi - zlo
            alpha = beta = gamma = 90.
        ts.dimensions = xlen, ylen, zlen, alpha, beta, gamma

        indices = np.zeros(self.n_atoms, dtype=int)

        f.readline()  # ITEM ATOMS etc
        for i in range(self.n_atoms):
            idx, _, xs, ys, zs = f.readline().split()

            indices[i] = idx
            ts.positions[i] = xs, ys, zs

        order = np.argsort(indices)
        ts.positions = ts.positions[order]
        # by default coordinates are given in scaled format, undo that
        if self.fractional:
            ts.positions = distances.transform_StoR(ts.positions, ts.dimensions)

        return ts
    

In [ ]:
def extract_trj_bug(simulation_pair, geometry, save_to):
    """
    extract_trj_bug does all the analysis on the a whole trajectory (simulation),

    Parameters:
    simulation_pair (a tuple of size 2): the pair of topology(first argument) and trajectory (second argument) of a simulation.
    geometry (string): the name of the geometry of the simulation box.
    save_to (str): address to which the output is saved

    Requirements:
    All the above-defined fucntions and classes, MDAnalysis.
    
    Caution:
    The histograms are for the cylindrical coordinate system in the geometry=cylindrical.
    For other goemteries you need to redfined them.
    """  
    print("This script for cylindrical geomery")
    print("Setting the name of analyze file...\n")
    today = datetime.date.today().strftime('%Y%m%d')
    cellAttrs = cellAttributes(simulation_pair[1],geometry)
    sim_name = cellAttrs.filename
    #ensGroup = "N"+str(cellAttrs.nmon)+'D'+str(cellAttrs.dcyl)+"_"
    outfile = save_to+sim_name+"-properties.csv"
    print("")
    print(sim_name+" is analyzing...")
    print("")
    with open(outfile, mode="w") as ensemble:
        ensemble.write(cellAttrs.cols)
        cylinder_write(ensemble,cellAttrs)

    print("Analyzing...")
    time_unit = 1.0 # time unit = dmon * sqrt(mmon * epmon)
    lj_nstep = cellAttrs.bdump # Sampling steps via dump command in Lammps
    lj_dt = cellAttrs.dt # Timestep during the sampling process in Lammps
    simulation_dt = lj_nstep * lj_dt * time_unit

    cell = mda.Universe(simulation_pair[0], simulation_pair[1], format = 'AMIRLAMMPSDUMP', atom_style = "id resid type x y z", dt = simulation_dt)
    chrm = cell.select_atoms('resid 1') # resid 1 is the atoms creating the chain
    print ("Caution:The histograms are for the cylindrical coordinate system in the geometry=cylindrical. For other goemteries you need to redfined them.")
    # radial direction of the cylindrical coordinate system
    edge_name = 'rEdges'
    bin_size = 0.1 * min(cellAttrs.dmon,cellAttrs.dcrowd)
    print(f"Bin size for r in the cylindrical geometry is set to 0.1*min(cellAttrs.dmon,cellAttrs.dcrowd)={bin_size} in a_m=1 units.")
    lmax = 0.5 * cellAttrs.dcyl
    redges, rhist_collectors = bin_ceate(sim_name, bin_size, 0.0, lmax, edge_name, save_to)

    # z direction of the cylindrical coordinate system
    edge_name = 'zEdges'
    bin_size = 0.5 * min(cellAttrs.dmon,cellAttrs.dcrowd)
    print(f"Bin size for z in the cylindrical geometry is set to 0.5*min(cellAttrs.dmon,cellAttrs.dcrowd)={bin_size} in a_m=1 units.")
    lmax = cellAttrs.lcyl / 2.
    zedges, zhist_collectors = bin_ceate(sim_name, bin_size, -1.0 * lmax, lmax, edge_name, save_to)

    # theta of the cylindrical coordinate system
    edge_name = 'thetaEdges'
    bin_size = np.degrees(np.pi/36) #bin size 5 degrees
    print(f"Bin size for theta in the cylindrical geometry is set to {bin_size} in degrees.")
    thetaedges, thetahist_collectors = bin_ceate(sim_name, bin_size, -1*np.degrees(np.pi), np.degres(np.pi), edge_name, save_to)

    # Chain end-to-end size distribution
    edge_name = 'rFloryEdges'
    lmax = cellAttrs.nmon * cellAttrs.dmon # The contour size of the chain
    bin_size = 0.5 * cellAttrs.dmon # in a_c=1.0 unit; for a_c<a_mon_small; check this
    print(f"Bin size for the PDF of end-to-end vector is set to 0.5*cellAttrs.dmon={bin_size} in a_m=1 units.")
    rFloryedges, rFloryhist_collectors = bin_ceate(sim_name, bin_size, 0, lmax, edge_name, save_to)

    #instantaneous quantity
    fsd_t = np.empty([0]) # the furthermost size using my own function fsd_cylinder.
    rFlory_t = np.empty([0]) # the end-to-end size, using my own function end_to_end.
    gyr_t = np.empty([0]) # radius of gyration

    if  any([rhist_collectors.any() != 0, zhist_collectors.any() != 0, thetahist_collectors.any() != 0, rFloryhist_collectors.any() != 0]):
        raise ValueError("One of the histogram collectors are not empty!")
    for ts in cell.trajectory: # the length of the for loop is equal to number of snapshots (configurations or frames)
        #chain size : the furthermost distance:
        fsd_t = np.append(fsd_t, np.array([fsd(chrm.positions)]), axis = 0)

        # radius of gyration
        gyr = chrm.radius_of_gyration()
        gyr_t = np.append(gyr_t, np.array([gyr]), axis = 0) # radius of gyration in each time step

        # end-to-end size distribution in eahc frame.
        rms = end_to_end(chrm.positions)
        rFlory_t = np.append(rFlory_t, np.array([rms]), axis = 0) # end-to-end vectors in each timestep

        # histogram in r direction
        dummy_hist, _ = np.histogram(rms, rFloryedges) # For one timestep, rms is equal to norm of end-to-end, so we use the later for histogram
        rFloryhist_collectors = np.add(rFloryhist_collectors, dummy_hist) 

        #number density in the cell's frame of reference
        # histogram in r direction
        rmon = np.linalg.norm(chrm.positions[:,:2], axis = 1) # r component of position of each monomer
        dummy_hist, _ = np.histogram(rmon, redges)
        rhist_collectors = np.add(rhist_collectors, dummy_hist)

        # histogram in z direction
        zmon = chrm.positions[:,2] # z component of position of each monomer
        dummy_hist, _ = np.histogram(zmon, zedges)
        zhist_collectors = np.add(zhist_collectors, dummy_hist)

        # histogram in theta 
        theta = np.degrees(np.arctan2(chrm.positions[:,1], chrm.positions[:,0])) # in degrees
        dummy_hist, _ = np.histogram(theta, thetaedges)
        thetahist_collectors = np.add(thetahist_collectors, dummy_hist)

    np.savetxt(save_to+sim_name+'-rHists.csv', rhist_collectors, delimiter = ',')
    np.savetxt(save_to+sim_name+'-zHists.csv', zhist_collectors, delimiter = ',')
    np.savetxt(save_to+sim_name+'-thetaHists.csv', thetahist_collectors, delimiter = ',')
    np.savetxt(save_to+sim_name+'-rFloryHists.csv', rFloryhist_collectors, delimiter = ',')
    np.savetxt(save_to+sim_name+'-fsd_t.csv', fsd_t, delimiter = ',')
    np.savetxt(save_to+sim_name+'-rFlory_t.csv', rFlory_t, delimiter = ',')
    np.savetxt(save_to+sim_name+'-gyr_t.csv', gyr_t, delimiter = ',')

    with open(outfile, mode="a") as ensemble:
        chain_stats(fsd_t, ensemble) # fsd,fsd_std,fsd_var,fsd_sem,
        chain_stats(rFlory_t, ensemble) # rflory,rflory_std,rflory_var,rflory_sem,
        chain_stats(gyr_t, ensemble) # gyr,gyr_std,gyr_var,gyr_sem,
        ensemble.write("{}\n".format(cell.trajectory.n_frames))
    print('done.') 

In [22]:
fname = glob("/Users/amirhsi_mini/ns*/ns*.bug.*")
fname = PipeLine.file_reader(fname) # This is a list with one member
fname

[('/Users/amirhsi_mini/ns700epss5.0epsb5.0r15.5dl5.0nl7ml125dc1.0nc97734lz113dt0.01bdump2000adump5000ens1ring/ns700epss5.0epsb5.0r15.5dl5.0nl7ml125dc1.0nc97734lz113dt0.01bdump2000adump5000ens1ring.bug.data',
  '/Users/amirhsi_mini/ns700epss5.0epsb5.0r15.5dl5.0nl7ml125dc1.0nc97734lz113dt0.01bdump2000adump5000ens1ring/ns700epss5.0epsb5.0r15.5dl5.0nl7ml125dc1.0nc97734lz113dt0.01bdump2000adump5000ens1ring.bug.lammpstrj')]

In [60]:
cell = mda.Universe(fname[0][0], fname[0][1], topology_format = 'DATA', format = 'LAMMPSDUMP', lammps_coordinate_convention= 'unscaled', atom_style = "id resid type x y z", dt = 0.01, velocities=True)
chrm = cell.select_atoms('resid 1') # resid 1 is the atoms creating the chain


In [76]:
chrm.velocities

NoDataError: This Timestep has no velocities

In [75]:
ts = cell.trajectory[0]
ts.velocities

NoDataError: This Timestep has no velocities